In [9]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import re

import pandas as pd
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


import pandas as pd
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [10]:
df = pd.read_csv('../data/merged/merged_airbnb_host.csv')

In [11]:
df['review_count_link'][2]

'https://www.airbnb.com/rooms/46789946/reviews?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-06-15&check_out=2024-06-21&source_impression_id=p3_1718221012_P3Ru5ryn0XNSs5_c&previous_page_section_name=1000&federated_search_id=a5fc4b6f-9e09-4174-be86-9e7b153c49bb'

In [12]:
df.dropna(subset='review_count_link',inplace=True)

In [13]:
df.shape

(533, 28)

In [14]:
df['review_count_link']

0       https://www.airbnb.com/rooms/88682598992554081...
1       https://www.airbnb.com/rooms/74127529237768681...
2       https://www.airbnb.com/rooms/46789946/reviews?...
5       https://www.airbnb.com/rooms/36290021/reviews?...
9       https://www.airbnb.com/rooms/42611763/reviews?...
                              ...                        
1409    https://www.airbnb.com/rooms/83957278587033329...
1411    https://www.airbnb.com/rooms/81860007359843078...
1417    https://www.airbnb.com/rooms/85485052403474440...
1418    https://www.airbnb.com/rooms/10895061749187734...
1419    https://www.airbnb.com/rooms/79165215919775820...
Name: review_count_link, Length: 533, dtype: object

In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

In [16]:
def scrape_review(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    try:
        # Open the URL
        driver.get(url)
        time.sleep(3)  # Wait for the page to load
        
        # Initialize a list to store the review data
        review_data = []

        # Find all reviews using XPath
        reviews = driver.find_elements(By.XPATH, "//div[@data-testid='pdp-reviews-modal-scrollable-panel']/div")
        
        for i, review in enumerate(reviews, start=1):
            review_dict = {
                'review_link': url,
                'reviewer_name': None,
                'reviewer_profile_link':None,
                'info': None,
                'rating_comment': None,
                'rating_extra': None
            }
            
            # Extract the necessary information
            try:
                reviewer_name_tag = review.find_element(By.XPATH, f"//div[@data-testid='pdp-reviews-modal-scrollable-panel']/div[{i}]/div/section/div/div/h2")
                reviewer_name = reviewer_name_tag.text if reviewer_name_tag else 'None'
                review_dict['reviewer_name'] = reviewer_name
            except Exception as e:
                print("User name not fetched", e)

            
            
            try:
                rating_extra_tag = review.find_element(By.XPATH, f"//div[@data-testid='pdp-reviews-modal-scrollable-panel']/div[{i}]/div/section/div/div/div")
                rating_extra = rating_extra_tag.text if rating_extra_tag else 'None'
                review_dict['rating_extra'] = rating_extra
            except Exception as e:
                print("No extra rating")


            try:
                 profile_link_elem = driver.find_element(By.XPATH,f"//a[@aria-label='{review_dict['reviewer_name']}']")
                                                                                  
                 review_dict['reviewer_profile_link'] = profile_link_elem.get_attribute('href')
            except:
                 print('reviewer profile link not found')
            
            try:
                  
                    parent_div = review.find_element(By.XPATH, './/div[@class="s78n3tv atm_c8_1w0928g atm_g3_1dd5bz5 atm_cs_9dzvea atm_9s_1txwivl atm_h_1h6ojuz dir dir-ltr"]')
                   
                    full_text = parent_div.get_attribute('textContent').strip()
                    
                    info_text = full_text.split('\n')[-1].strip()

                    review_dict['info'] = info_text

            except:
                    print("review_time not fetched")

            try:
                rating_comment_tag = review.find_element(By.XPATH, f"(.//div[@class='r1bctolv atm_c8_1sjzizj atm_g3_1dgusqm atm_26_lfmit2_13uojos atm_5j_1y44olf_13uojos atm_l8_1s2714j_13uojos dir dir-ltr']/div/span/span)")
                rating_comment = rating_comment_tag.text if rating_comment_tag else 'N/A'
                review_dict['rating_comment'] = rating_comment
            except Exception as e:
                print("Rating comment not fetched", e)

            # Append the extracted data to the review_data list
            review_data.append(review_dict)

        driver.quit()
        return review_data

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        driver.quit()
        return []

# Loop through each URL in the DataFrame and scrape review information

k=1
for url in df['review_count_link']:
    reviews = scrape_review(url)
    

    review_df = pd.DataFrame(reviews)
    review_df.to_csv(f'../data/reviews_csvs/airbnb_reviews_{k}.csv')
    k+=1
    
    



User name not fetched Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@data-testid='pdp-reviews-modal-scrollable-panel']/div[25]/div/section/div/div/h2"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7907F1F52+60322]
	(No symbol) [0x00007FF79076CEC9]
	(No symbol) [0x00007FF790627EBA]
	(No symbol) [0x00007FF790677676]
	(No symbol) [0x00007FF79067773C]
	(No symbol) [0x00007FF79066AEEC]
	(No symbol) [0x00007FF79069C25F]
	(No symbol) [0x00007FF79066ADB6]
	(No symbol) [0x00007FF79069C430]
	(No symbol) [0x00007FF7906BBC80]
	(No symbol) [0x00007FF79069BFC3]
	(No symbol) [0x00007FF790669617]
	(No symbol) [0x00007FF79066A211]
	GetHandleVerifier [0x00007FF790B094AD+3301629]
	GetHandleVerifier [0x00007FF790B536D3+3605283]
	GetHandleVerifier [0x00007FF790B49450+3563680]
	GetHandle